# [Module 3] Training with Pipe Mode using PipeModeDataset

Amazon SageMaker를 사용하면 Pipe 입력 모드를 사용하여 교육 작업을 생성할 수 있습니다. **Pipe 입력 모드를 사용하면 S3의 학습 데이터셋을 노트북 인스턴스의 로컬 디스크로 다운로드하는 대신 학습 인스턴스로 직접 스트리밍합니다.** 즉, 학습 작업이 더 빨리 시작되고 더 빨리 완료되며 더 적은 디스크 공간이 필요합니다.

SageMaker TensorFlow는 SageMaker에서 Pipe 입력 모드를 쉽게 활용할 수있는 `tf.data.Dataset`의 구현을 제공합니다. `tf.data.Dataset`을`sagemaker_tensorflow.PipeModeDataset`으로 바꾸면 학습 인스턴스로 스트리밍되는 TFRecord를 읽을 수 있습니다.

여러분의 entry_point 스크립트에서 `PipeModeDataset`을 `Dataset`처럼 사용할 수 있습니다. 아래 예시는 '학습' 채널에서 TFRecords를 읽을 수있는`PipeModeDataset`을 생성하는 예시입니다.

```python
from sagemaker_tensorflow import PipeModeDataset

features = {
    'data': tf.FixedLenFeature([], tf.string),
    'labels': tf.FixedLenFeature([], tf.int64),
}

def parse(record):
    parsed = tf.parse_single_example(record, features)
    return ({
        'data': tf.decode_raw(parsed['data'], tf.float64)
    }, parsed['labels'])

def train_input_fn(training_dir, hyperparameters):
    ds = PipeModeDataset(channel='training', record_format='TFRecord')
    ds = ds.repeat(20)
    ds = ds.prefetch(10)
    ds = ds.map(parse, num_parallel_calls=10)
    ds = ds.batch(64)
    return ds
```

Pipe 입력 모드에서 학습 작업을 구동하려면, 아래 예시처럼 여러분의 TensorFlow Estimator에서 `input_mode='Pipe'` 인자를 추가해 주세요.

```python
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='tf-train-with-pipemodedataset.py', role='SageMakerRole',
                          train_instance_count=1, train_instance_type='ml.c5.2xlarge',
                          framework_version='1.14.0', input_mode='Pipe')

tf_estimator.fit('s3://bucket/path/to/training/data')
```

## Create a training script that support pipemode datasets
**`training_script/cifar10_keras_sm.py`에서 스크립트 사본을 만들어 `training_script/cifar10_keras_pipe.py`로 저장하세요.**

스크립트 사본을 생성하였다면 단계별로 아래의 작업들을 직접 시도합니다.

----
### TODO 1.
`cifar10_keras_pipe.py`에서 아래와 같이 `PipeModeDataset`를 import해 주세요.
```python
from sagemaker_tensorflow import PipeModeDataset
```
----
### TODO 2.
```python
def _input(epochs, batch_size, channel, channel_name):
```
함수 내에서
```python
dataset = tf.data.TFRecordDataset(filenames)
```
을
```python
dataset = PipeModeDataset(channel=channel_name, record_format='TFRecord')
```
로 수정해 주세요.

자세한 내용은 SageMaker-python-sdk [documentation](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-pipe-mode-using-pipemodedataset)를 참조해 주세요.

<font color='blue'>**본 노트북 실습에 어려움이 있다면 솔루션 파일 `training_script/cifar10_keras_pipe_solution.py`을 참조하시면 됩니다.**</font>

In [4]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [5]:
prefix = 'data/DEMO-cifar10'
dataset_location = os.path.join('s3://', sagemaker_session.default_bucket(), prefix)

`input_mode='Pipe'`로 Pipe Mode를 활성화한 후에 10 epoch를 학습해 보세요.
모든 작업에 metric_definitions 인수를 추가해야 합니다.<br>
또한, AWS 콘솔 가시성을 위해 `base_job_name`을 `'cifar10-pipe'`로 설정해 주세요.

In [6]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

In [7]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10-pipe',
                       entry_point='cifar10_keras_pipe.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.14.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge', 
                       metric_definitions=metric_definitions, # 1_Monitoring_your_TensorFlow_scripts.ipynb 참조                       
                       input_mode='Pipe' # 추가
                      )

In [8]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2020-02-28 06:26:35 Starting - Starting the training job...
2020-02-28 06:26:36 Starting - Launching requested ML instances...
2020-02-28 06:27:33 Starting - Preparing the instances for training.........
2020-02-28 06:29:05 Downloading - Downloading input data
2020-02-28 06:29:05 Training - Downloading the training image......../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: F

**잘 하셨습니다.** 

본 실습에서 `PipeModeDataset`을 정의해 학습해 보았습니다. 큰 데이터셋에서 Pipe 모드를 사용하면 학습 시간이 감소하고 노트북 인스턴스의 로컬 디스크 공간도 절약할 수 있습니다. <br>
다음 노트북으로 계속 진행하기 전에 CloudWatch 및 TensorBoard의 Pipe 모드 작업 지표를 살펴보세요.